# 🧠 Interactive Hodgkin-Huxley Interactive Simulation

This notebook simulates the classic Hodgkin-Huxley model of the action potential in neurons.

- Adjust the **injected current** to see how the neuron responds
- Explore how ion channels (Na⁺, K⁺) shape the spike
- Gating variables m, h, n control activation/inactivation


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

%matplotlib inline


### 🔬 Model Parameters (Biological Constants)

- `g_Na`, `g_K`, and `g_L` control how easily each ion flows through the membrane.
- `E_Na`, `E_K`, and `E_L` are the voltages each ion "wants" to reach.

In [2]:
# Membrane capacitance
C_m  = 1.0   # uF/cm²

# Maximum conductances (how leaky each channel is when open)
g_Na = 120.0  # Sodium (Na⁺)
g_K  = 36.0   # Potassium (K⁺)
g_L  = 0.3    # Leak (mainly Cl⁻)

# Reversal potentials (in mV)
E_Na = 50.0    # Where Na⁺ wants to go
E_K  = -77.0   # Where K⁺ wants to go
E_L  = -54.387 # Leak current equilibrium

## 🔁 Gating Variable Functions

Each ion channel opens and closes based on voltage. These α (alpha) and β (beta) functions describe the opening and closing **rates** of:
- `m`: Na⁺ activation
- `h`: Na⁺ inactivation
- `n`: K⁺ activation

They follow first-order differential equations.

In [3]:
def alpha_n(V): return 0.01*(V + 55)/(1 - np.exp(-(V + 55)/10))
def beta_n(V): return 0.125*np.exp(-(V + 65)/80)

def alpha_m(V): return 0.1*(V + 40)/(1 - np.exp(-(V + 40)/10))
def beta_m(V): return 4.0*np.exp(-(V + 65)/18)

def alpha_h(V): return 0.07*np.exp(-(V + 65)/20)
def beta_h(V): return 1/(1 + np.exp(-(V + 35)/10))

## 🧮  Simulating the Neuron

This function simulates how the membrane potential changes over time in response to a constant input current (`I_inj`).

Each timestep:
- Gating variables (`m`, `h`, `n`) are updated
- Ion currents (`I_Na`, `I_K`, `I_L`) are calculated
- Membrane voltage (`V`) is updated using the total current

We use the Euler method for numerical integration.

In [4]:
def simulate(I_inj=10.0, t_max=50, dt=0.01):
    time = np.arange(0, t_max, dt)
    n_steps = len(time)

    # Initialize state variables
    V = np.full(n_steps, -65.0)
    m = np.zeros(n_steps)
    h = np.zeros(n_steps)
    n = np.zeros(n_steps)

    # Initial gating values
    m[0] = alpha_m(V[0]) / (alpha_m(V[0]) + beta_m(V[0]))
    h[0] = alpha_h(V[0]) / (alpha_h(V[0]) + beta_h(V[0]))
    n[0] = alpha_n(V[0]) / (alpha_n(V[0]) + beta_n(V[0]))

    # Currents for plotting
    I_Na = np.zeros(n_steps)
    I_K = np.zeros(n_steps)
    I_L = np.zeros(n_steps)

    for i in range(1, n_steps):
        # Update gating variables
        m[i] = m[i-1] + dt * (alpha_m(V[i-1]) * (1 - m[i-1]) - beta_m(V[i-1]) * m[i-1])
        h[i] = h[i-1] + dt * (alpha_h(V[i-1]) * (1 - h[i-1]) - beta_h(V[i-1]) * h[i-1])
        n[i] = n[i-1] + dt * (alpha_n(V[i-1]) * (1 - n[i-1]) - beta_n(V[i-1]) * n[i-1])

        # Compute currents
        I_Na[i] = g_Na * m[i]**3 * h[i] * (V[i-1] - E_Na)
        I_K[i]  = g_K  * n[i]**4 * (V[i-1] - E_K)
        I_L[i]  = g_L  * (V[i-1] - E_L)

        # Total current
        I_ion = I_inj - I_Na[i] - I_K[i] - I_L[i]

        # Update voltage
        V[i] = V[i-1] + dt * I_ion / C_m

    return time, V, m, h, n, I_Na, I_K, I_L


## 📊 Plotting

We visualize:
1. The changing **membrane voltage** over time
2. The **gating variables** that control ion channels
3. The individual **ion currents**

This helps us understand how a neuron's action potential is generated from microscopic channel dynamics.

In [19]:
def plot_hh(I_inj):
    time, V, m, h, n, I_Na, I_K, I_L = simulate(I_inj=I_inj)

    fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)

    axs[0].plot(time, V, label='Membrane Potential (mV)')
    axs[0].set_ylabel('V (mV)')
    axs[0].legend()

    axs[1].plot(time, m, label='m')
    axs[1].plot(time, h, label='h')
    axs[1].plot(time, n, label='n')
    axs[1].set_ylabel('Gating Variables')
    axs[1].legend()

    axs[2].plot(time, I_Na, label='I_Na')
    axs[2].plot(time, I_K, label='I_K')
    axs[2].plot(time, I_L, label='I_L')
    axs[2].set_ylabel('Currents (uA/cm²)')
    axs[2].set_xlabel('Time (ms)')
    axs[2].legend()

    plt.tight_layout()
    plt.show()


## 🎛️ Try It Yourself!

Use the slider to change the input current and observe how the neuron's behavior changes.

- Small current: no spike
- Medium current: single spike
- High current: multiple spikes (repetitive firing)

In [20]:
interact(plot_hh, I_inj=FloatSlider(min=0, max=50, step=1, value=10, description='Injected Current'))

interactive(children=(FloatSlider(value=10.0, description='Injected Current', max=50.0, step=1.0), Output()), …

<function __main__.plot_hh(I_inj)>